# Using SqlAlchemy ORM

## Get the db connection info
### This section can go in a db_setup.py module

In [1]:
# Get Database URI from appsettings

import json

settings = json.loads(open('appsettings.json').read())

db_config = settings['db_config']
db_url = "mysql+pymysql://%s:%s@%s/%s" % (db_config['user'],
    db_config['password'],
    db_config['host'],
    db_config['database'])


## All the imports and initial DB setup
Note that this code will even create the database for you if it doesn't exist. Also note that this code won't run unless you have pip installed sqlalchemy and sqlalchemy_utils in your system python (specifically, the jupyter/anaconda python running this notebook). This could also go in the db_setup module -- engine and Base need to be available to the models.

In [2]:
from sqlalchemy import (
    create_engine,
    inspect,
    Column,
    ForeignKey,
    String,
    Integer,
    Numeric,
    DateTime )

from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy_utils import database_exists, create_database

# Create the database if it doesn't exist
engine = create_engine(db_url)
if not database_exists(engine.url):
    create_database(engine.url)

Base = declarative_base()

## Now, let's define our classes/tables without SQL!
And we are making using of SQLAlchemy's relationship function which magically adds fields to the class to make accessing related object/tables much easier.

In [3]:
class Stream(Base):
    __tablename__ = 'streams'
    id = Column(Integer, primary_key=True)
    name = Column(String(64))
    description = Column(String(255))
    sites = relationship("Site", back_populates="stream")

class Site(Base):
    __tablename__ = 'sites'
    id = Column(Integer, primary_key=True)
    name = Column(String(64))
    latitude = Column(Numeric(3,3))
    longitude = Column(Numeric(3,3))
    stream_id = Column(Integer, ForeignKey('streams.id'))
    stream = relationship("Stream", back_populates="sites")

## Let SqlAlchemy create the tables for you!

In [4]:
Base.metadata.create_all(bind=engine)

# check table exists
ins = inspect(engine)
for _t in ins.get_table_names(): print(_t)

sites
streams


## RDBMS interaction without SQL!
This is a more natural way for an application developer to interact with a database -- asside from session, it is similar to dealing with in memory objects.

### Insert some data

In [5]:
# We do need to get a Session from the engine to which we are bound.
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

# And from that Session, we can create session instances to control db operations
session = Session()

james = Stream(name="James")
potomac = Stream(name="Potomac")

maidens = Site(name="Maidens")
maidens.stream = james

cartersville = Site(name="Cartersville")
cartersville.stream = james

occoquan = Site(name="Occoquan")
occoquan.stream = potomac

# Prep the in memory objects to be saved to the db.
session.add_all([james,potomac,maidens,occoquan])

# Commit will actually save all objects bound to the session
session.commit()


### Query the DB

In [6]:
session = Session()

for site in session.query(Site).order_by(Site.name):
    print(site.id,site.name,site.stream.name)
    
print()

for stream in session.query(Stream).order_by(Stream.name):
    print(stream.name,':')
    for site in stream.sites:
        print('->',site.name)
    

2 Cartersville James
1 Maidens James
3 Occoquan Potomac

James :
-> Maidens
-> Cartersville
Potomac :
-> Occoquan
